In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델 로드
model_name = "Chanjeans/gamerecommend"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="./model_cache")
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir="./model_cache").to("cuda")  # GPU 사용


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/7.38M [00:00<?, ?B/s]

In [ ]:
import re
from transformers import pipeline

# Summarization 파이프라인 로드
summarizer = pipeline("summarization")


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
def recommend_game(user_input, model, tokenizer, summarizer):
    # 개선된 프롬프트
    prompt = "\nRecommend me at least three games about what user_input.user_input : "+user_input+". Also provide descriptions for me to understand about each games and why you recommended it specifically.\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # 모델 생성
    outputs = model.generate(
        inputs["input_ids"],
        max_new_tokens=1024,
        num_beams=5,
        early_stopping=True,
        no_repeat_ngram_size=2,
        temperature=0.7,
        top_p=0.9
    )

    # 출력 디코딩 및 디버깅
    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Model Output:", raw_output)

    # 프롬프트 부분 제거
    output_text = raw_output[len(prompt):].strip()

    # 추천 게임 목록 추출
    recommendations = re.findall(r'- (.+?)(?=\n-|\Z)', output_text, re.DOTALL)

    result = []
    for rec in recommendations[:3]:  # 최대 3개 제한
        if ': ' in rec:
            title, description = rec.split(': ', 1)
        elif '\n' in rec:
            title, description = rec.split('\n', 1)
        else:
            title = rec
            description = ''

        title = title.strip()
        description = description.strip()

        # 설명 요약
        if description:
            summary = summarizer(description, max_length=50, min_length=25, do_sample=False)[0]['summary_text']
        else:
            summary = ''
        result.append((title, summary))

    return result


In [ ]:
# 사용자 입력 받기
user_input = input("Enter your game perference: ")

# 추천 함수 호출

recommendations = recommend_game(user_input, model, tokenizer,summarizer)

Enter your game perference: Can you find me a fun Arcade, Racing game?
Model Output: 
Recommend me at least three games about what user_input.user_input : Can you find me a fun Arcade, Racing game?. Also provide descriptions for me to understand about each games and why you recommended it specifically.
Genre: Racing, Simulation
Release Date: 2023-09-13
Platform: PC, PlayStation 4, Xbox One
Genres: Adventure, RPG, Shooter, Strategy, Tactical, Turn Based Strategy
Description: Turn based strategy game where the player takes on the role of a knight in a quest to save the kingdom from the evil sorcerer Mordred. Use your knightly skills and magical abilities to defeat the sorcerer and restore peace to the land. Turn-based strategy games are a type of video game in which players control a character or group of characters who take turns to move, attack, and interact with the game world. The player can control one character at a time, or control multiple characters at the same time. This genre 

In [ ]:
# 사용자 입력 받기
user_input = input("Enter your game perference: ")

# 추천 함수 호출
recommendations = recommend_game(user_input, model, tokenizer,summarizer)

# 결과 출력




Enter your game perference: Can you find me a fun Arcade, Racing game?
Model Output: 
Recommend me three games about what user_input.user_input : Can you find me a fun Arcade, Racing game?. Also provide descriptions for me to understand about each games and why you recommended it specifically.
Game 1: Mario Kart 8 Deluxe
Mario Kart is a racing game in which players control characters driving karts on a variety of tracks. The goal of the game is to be the first player to cross the finish line. Players can also use items to help them win the race, such as power-ups that give them a speed boost or the ability to jump over obstacles. There are a total of 70 different characters to choose from, each with their own unique abilities. Mario, Luigi, Peach, Yoshi, Toad, Wario, Waluigi, and Rosalina are the only characters that can be used in the single-player mode, while the other characters are available in all three modes: single player, co-op and online multiplayer.

 Game 2: Forza Horizon 5


In [ ]:
# 사용자 입력 받기
user_input = input("Enter your game perference: ")

# 추천 함수 호출
recommendations = recommend_game(user_input, model, tokenizer,summarizer)



Enter your game perference: peaceful animals or relaxing game
Model Output: 
Recommend me at least three games about what user_input.user_input : peaceful animals or relaxing game. Also provide descriptions for me to understand about each games and why you recommended it specifically.
Game 1: Animal Crossing: New Horizons

Description: Explore a vibrant open world filled with endless possibilities. Build your dream home, raise adorable animals, and create your own island paradise.

Features:
- Customize your character with a variety of outfits, hairstyles, furniture, decorations, tools, vehicles, pets and more.


- Interact with the world around you by talking to villagers, fishing, hunting, mining, gardening, cooking, crafting, decorating, playing music, dancing, attending events, going to the beach, swimming, surfing, kitesurfing, windsurfing, kiteboarding, paddle boarding, kayaking, canoeing and other activities.



- Decorate your home with items you collect around the island and b